# Imports

In [1]:
import numpy as np
import heapq
import pandas as pd

## Constructing a matrix representing the layout

In [2]:
def divide_aisles(nbMidAisles):
    
    ''' 
    Function which defines the position of the middle aisles 
    depending on how many are required.
    '''
    if nbMidAisles == 0:
        return [(5,29)]
    elif nbMidAisles == 1:
        return [(4,16), (17,29)]
    elif nbMidAisles == 2:
        return [(3,11), (12,20), (21,29)]
    elif nbMidAisles == 3:
        return [(2,8), (9,15), (16,22), (23,29)]
    elif nbMidAisles == 4:
        return [(1,5), (6,11), (12,17), (18,23), (24,29)]

def construct_warehouse_layout(nbMidAisles):

    '''
    Function which takes the number of middle aisles required and 
    returns a warehouse layout with the added aisles in the warehouse.
    '''

    # Define the shape of the grid
    rows = 30
    cols = 13

    filled_squares = [] # initialise shelves to fill

    # define where to place the middle aisles and add shelves accordingly
    row_ranges = divide_aisles(nbMidAisles)
    columns = [1,3,4,6] # columns to add the shelves to
    for start, end in row_ranges:
        for row in range(start, end):
            for col in columns:
                filled_squares.append((row, col))

    # Initialize the grid as zeros
    grid = np.zeros((rows, cols), dtype=int)

    # Mark filled squares as 1
    for row,col in filled_squares:
        grid[row, col] = 1

    # Find indices that define the corridors between columns of shelves
    adjacent_shelves = []
    for i in range(len(grid)):
        # Iterate over each column, skipping the first and last columns
        for j in range(1, len(grid[i]) - 1):
            # Check if current element is '0' and both its neighbors are '1'
            if grid[i][j] == 0 and grid[i][j - 1] == 1 and grid[i][j + 1] == 1:
                adjacent_shelves.append([i, j])  # Append indices to the list
    return grid, adjacent_shelves

## $\text{A}^*$ Algorithm
#### This is a pathfinding algorithm which calculates the Manhattan distance between points while avoiding obstacles (shelves).

In [3]:
class Shelf:
    def __init__(self, x, y, parent=None):
        # initialising shelf object with coordinates and an optional parent shelf
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0 # cost from the door to this shelf
        self.h = 0 # cost from the shelf to the destination

    def __lt__(self, other):
        # comparison method for priority queue
        return (self.g + self.h) < (other.g + other.h)

def get_neighbours(shelf, warehouse_grid):
    # obtain neighbouring shelves accessible from the current shelf
    neighbours = []
    moves = [(1, 0), (-1, 0), (0, 1), (0, -1)] # define the possible movements
    for dx, dy in moves:
        x, y = shelf.x + dx, shelf.y + dy
        # check neighbouring coordinates are within warehouse boundaries and unblocked
        if 0 <= x < len(warehouse_grid) and 0 <= y < len(warehouse_grid[0]) and warehouse_grid[x][y] == 0:
            neighbours.append(Shelf(x, y, shelf))
    return neighbours

def manhattan_distance(shelf1, shelf2):
    # calculate Manhhattan distance between two shelves
    return abs(shelf1.x - shelf2.x) + abs(shelf1.y - shelf2.y)

def a_star(start, destination, warehouse_grid):
    '''
    Function uses the A* algorithm to find the shortest path 
    from the current shelf to the destination shelf.
    '''
    open_list = [] # initialise priority queue of shelves to be evaluated
    closed_set = set() # the set of evaluated shelves

    heapq.heappush(open_list, start) # add starting shelf to priority queue

    while open_list:
        current_shelf = heapq.heappop(open_list)
        # checking if the current shelf is the destination
        if (current_shelf.x, current_shelf.y) == (destination.x, destination.y):
            path = []

            # reconstruct path by tracing back from destination to the start
            while current_shelf:
                path.append((current_shelf.x, current_shelf.y))
                current_shelf = current_shelf.parent
            return path[::-1] # reverse path to rewrite at start -> destination

        closed_set.add((current_shelf.x, current_shelf.y)) # add current shelf to the closed set

        # Exploring neighbours of the current shelf
        for neighbour in get_neighbours(current_shelf, warehouse_grid):
            if (neighbour.x, neighbour.y) in closed_set:
                continue
            # calculate tentative g and h scores for the neighbour
            neighbour.g = current_shelf.g + 1
            neighbour.h = manhattan_distance(neighbour, destination)
            heapq.heappush(open_list, neighbour) # add neighbour to the open list

    return None # if no path is found

def calculate_path_and_distance(start, end, warehouse_grid):
    # calculate path and total distance between start and end shelves
    path = a_star(Shelf(*start), Shelf(*end), warehouse_grid)
    if path:
        # calculate total distance by summing distances between shelves along the path
        total_distance = sum(manhattan_distance(Shelf(*path[i]), Shelf(*path[i+1])) for i in range(len(path)-1))
        return path, total_distance
    else:
        return None, None # if no path is found, return None for path and distance

## Using $\text{A}^*$ to construct the distance matrix

In [4]:
def construct_distance_matrix(warehouse_layout, shelf_positions):
    ''' 
    Function uses the A* algorithm to calculate pairwise distances between shelves. 

    It only calculates half of the shelves as distances are repeated for each column.

    Hence, a new full distance matrix is finally constructed.
    '''
    # sorting the shelves by columns, as in the floor plan
    shelf_positions = sorted(shelf_positions, key = lambda shelf: (shelf[1], -shelf[0]))

    # initialise distance matrix (48x48)
    distance_matrix = np.zeros((len(shelf_positions), len(shelf_positions)), dtype = int)
    paths = {} # intiialise dict to store paths

    # find pairwise distances and add to distance matrix
    for i, start_point in enumerate(shelf_positions):
        for j, end_point in enumerate(shelf_positions):
            path, total_distance = calculate_path_and_distance(start_point, end_point, warehouse_layout)
            distance_matrix[i,j] = total_distance
            paths[f"{start_point} to {end_point}"] = path

        # optional code to print distances and paths
            # if path:
            #     print(f"Path from {start_point} to {end_point}:", path, total_distance)
            #     # print("Total Manhattan Distance:", total_distance)
            # else:
            #     print("No path found.")
            
    quad_quad_mat = np.zeros((96,96), dtype = int) # initialise full distance matrix

    # find the quadrants to be repeated
    TL_quad = distance_matrix[:24, :24]
    TR_quad = distance_matrix[:24, -24:]
    BL_quad = distance_matrix[-24:, :24]
    BR_quad = distance_matrix[-24:, -24:]

    # assign quadrant values to the full distance matrix
    quad_quad_mat[:24, :24] = quad_quad_mat[:24, 24:48] = quad_quad_mat[24:48, :24] = quad_quad_mat[24:48, 24:48] = TL_quad

    quad_quad_mat[:24, 48:72] = quad_quad_mat[:24, 72:] = quad_quad_mat[24:48, 48:72] = quad_quad_mat[24:48, 72:] = TR_quad

    quad_quad_mat[48:72, :24] = quad_quad_mat[48:72, 24:48] = quad_quad_mat[72:, :24] = quad_quad_mat[72:, 24:48] = BL_quad

    quad_quad_mat[48:72, 48:72] = quad_quad_mat[48:72, 72:] = quad_quad_mat[72:, 48:72] = quad_quad_mat[72:, 72:] = BR_quad

    return quad_quad_mat, paths # return full distance matrix and paths

In [5]:
def construct_door_shelf_distances(warehouse_layout, shelf_positions, door_position = [29, 3]):
    ''' 
    Function takes the postiion of the door and calculates a 
    distance vector for each shelf.

    Since distances are based on corridors and not shelves themselves, the 
    vector is split and repeated to give the full distance vector.
    '''
    # sorting shelves by columns, as in the floor plan
    shelf_positions = sorted(shelf_positions, key = lambda shelf: (shelf[1], -shelf[0]))
    init_door_dist = np.zeros(len(shelf_positions) + 1, dtype = int) # initialise empty distance vector
    paths = {} # initialise dict of paths from door to shelves

    # calculate distances between door and each shelf
    for i, shelf in enumerate(shelf_positions):
        path, total_distance = calculate_path_and_distance(door_position, shelf, warehouse_layout)
        init_door_dist[i+1] = total_distance
        paths[f"Door to {shelf}"] = path

    # optional code to print paths and distances
        # if path:
        #     print(f"Path from door to {shelf}:", path, total_distance)
        # else:
        #     print("No path found.")
    
    # divide distance vector to obtain the different sets of values to repeat
    first_half = init_door_dist[1:25]
    second_half = init_door_dist[25:]

    # create full distance vector with door and repeat values as required
    door_shelf_dist = np.zeros(len(shelf_positions*2) + 1, dtype = int)
    door_shelf_dist[1:25] = door_shelf_dist[25:49] = first_half
    door_shelf_dist[49:73] = door_shelf_dist[73:] = second_half

    return door_shelf_dist, paths

## Writing distance matrices (squares and metres) to Excel files

In [6]:
for i in range(5):
    warehouse_layout, adjacent_shelves = construct_warehouse_layout(i)
    distance_mat = construct_distance_matrix(warehouse_layout, adjacent_shelves)[0]
    door_shelf_dist_vec = construct_door_shelf_distances(warehouse_layout, adjacent_shelves)[0]

    full_distance_matrix_squares = np.zeros((len(door_shelf_dist_vec), len(door_shelf_dist_vec)), dtype = int)
    full_distance_matrix_squares[0, 1:] = full_distance_matrix_squares[1:, 0] = door_shelf_dist_vec[1:]
    full_distance_matrix_squares[1:, 1:] = distance_mat

    full_distance_matrix_metres = full_distance_matrix_squares*3

    full_distance_matrix_squares_df = pd.DataFrame(full_distance_matrix_squares)
    full_distance_matrix_metres_df = pd.DataFrame(full_distance_matrix_metres)

    with pd.ExcelWriter(f"Distance_Matrices/DistanceMatrix_{i}_aisles.xlsx") as writer:  
        full_distance_matrix_squares_df.to_excel(writer, sheet_name="DistanceMatrixSquares", index = False)
        full_distance_matrix_metres_df.to_excel(writer, sheet_name="DistanceMatrixMetres", index = False)